# Correr experimentos
En este archivo está el código para correr los experimentos y escribir los resultados en archivos CSV.
> Los archivos se guardan en la carpeta _resultados_.

In [1]:
import math, subprocess
import pandas as pd
import numpy as np
from IPython.display import display, clear_output

A continuación leemos los datasets en dataframes de Pandas.

In [28]:
def leer_instancia(path_instancia):
    with open(path_instancia, "r") as f:
        return f.read();

In [63]:
df_mejor_caso_bt_f = pd.read_csv("instancias/mejor-caso-bt-f/indice.csv");

In [65]:
df_mejor_caso_bt_f

,dataset,instancia,n,R,archivo
0,mejor-caso-bt-f,BT-MC-F-1,1,1000,instancias/mejor-caso-bt-f/BT-MC-F-1.txt
1,mejor-caso-bt-f,BT-MC-F-2,2,1000,instancias/mejor-caso-bt-f/BT-MC-F-2.txt
2,mejor-caso-bt-f,BT-MC-F-3,3,1000,instancias/mejor-caso-bt-f/BT-MC-F-3.txt
3,mejor-caso-bt-f,BT-MC-F-4,4,1000,instancias/mejor-caso-bt-f/BT-MC-F-4.txt
4,mejor-caso-bt-f,BT-MC-F-5,5,1000,instancias/mejor-caso-bt-f/BT-MC-F-5.txt
...,...,...,...,...,...
994,mejor-caso-bt-f,BT-MC-F-995,995,1000,instancias/mejor-caso-bt-f/BT-MC-F-995.txt
995,mejor-caso-bt-f,BT-MC-F-996,996,1000,instancias/mejor-caso-bt-f/BT-MC-F-996.txt
996,mejor-caso-bt-f,BT-MC-F-997,997,1000,instancias/mejor-caso-bt-f/BT-MC-F-997.txt
997,mejor-caso-bt-f,BT-MC-F-998,998,1000,instancias/mejor-caso-bt-f/BT-MC-F-998.txt


In [64]:
df_peor_caso_bt_f = pd.read_csv("instancias/peor-caso-bt-f/indice.csv");

In [4]:
df_dinamica = pd.read_csv("instancias/dinamica/indice.csv");

In [5]:
df_bt= pd.read_csv("instancias/bt/indice.csv");

In [6]:
df_fbms = pd.read_csv("instancias/muchas-soluciones-validas/indice.csv");
df_fbps = pd.read_csv("instancias/pocas-soluciones-validas/indice.csv");

La siguiente función sirve para correr el código sobre una instancia ejecutando un método en particular.
- FB: Fuerza bruta
- BT: Backtracking con ambas podas.
- BT-F: Backtracking solamente con poda por factibilidad.
- BT-O: Backtracking solamente con poda por optimalidad.
- PD: Programación dinámica.

In [8]:
def correr_experimento(metodo, archivo_instancia):
    # Leer archivo de la instancia.
    instancia = leer_instancia(archivo_instancia)
    
    # Crear proceso para ejecutar el codigo.
    process = subprocess.Popen(["../main", metodo], stderr=subprocess.PIPE, stdout=subprocess.PIPE, stdin=subprocess.PIPE, universal_newlines = True)

    # Poner la instancia en la entrada estandar.
    process.stdin.write(instancia)
    process.stdin.flush()

    # Correr experimento.
    exit_code = process.wait()

    # Verificar que el proceso no fallo.
    if exit_code != 0: raise(F"Hubo un error en la experimentacion para el algoritmo: {algoritmo} con la instancia {archivo_instancia}.")
    # Leer salida de STDERR con los tiempos de ejecucion de cada metodo.
    tiempo_de_ejecucion = float(process.stderr.read());
    
    process.stdin.close();
    process.stdout.close();
    process.stderr.close();
    
    return tiempo_de_ejecucion;

## Corremos los experimentos
Vamos a guardar una tabla con las ejecuciones y sus respectivos tiempos.

In [79]:
experimentos = [];

## Experimento Fuerza Bruta : Pocas/Muchas soluciones

In [47]:
for i in range(0, df_fbms.shape[0]):
    fila = df_fbms.iloc[i];
    n = fila["n"] 
    if (n <= 25):
        experimentos.append([fila["dataset"], fila["n"], fila["R"], "FB", fila["archivo"]]);
    
for i in range(0, df_fbps.shape[0]):
    fila = df_fbps.iloc[i];
    n = fila["n"] 
    if (n <= 25):
        experimentos.append([fila["dataset"], fila["n"], fila["R"], "FB", fila["archivo"]]);

## Experimento Back Tracking - Mejor caso poda por factibilidad == Peor caso optimalidad


In [48]:
# mejor caso factibilidad

for n in range(0, df_mejor_caso_bt_f.shape[0]):
    fila_n = df_mejor_caso_bt_f.iloc[n];
    experimentos.append([fila_n["dataset"], n, fila_n["R"], "BT-F", fila_n["archivo"]]);

In [80]:
# peor caso optimalidad
for n in range(0, df_mejor_caso_bt_f.shape[0]):
    fila_n = df_mejor_caso_bt_f.iloc[n];
    tam = fila_n["n"] 
    if (tam < 30):
        experimentos.append([fila_n["dataset"], n, fila_n["R"], "BT-O", fila_n["archivo"]]);

## Experimento Back Tracking - Peor caso Factibilidad

In [81]:
#peor caso factibilidad
for n in range(0, df_peor_caso_bt_f.shape[0]):
    
    fila_n = df_peor_caso_bt_f.iloc[n];
    tam = fila_n["n"] 
    if (tam < 30):
        experimentos.append([fila_n["dataset"], n, fila_n["R"], "BT-F", fila_n["archivo"]]);

In [62]:
#mejor caso optimalidad
for n in range(0, df_peor_caso_bt_f.shape[0]):
    fila_n = df_peor_caso_bt_f.iloc[n];
    experimentos.append([fila_n["dataset"], n, fila_n["R"], "BT-O", fila_n["archivo"]]);

## Experimento Campana de Back Tracking

In [52]:
for i in range(0, df_bt.shape[0]):
    fila = df_bt.iloc[i];
    experimentos.append([fila["dataset"], fila["n"], fila["R"], "BT", fila["archivo"]]);

## Experimento Programacion Dinamica


In [53]:
for i in range(0, df_dinamica.shape[0]):
    fila = df_dinamica.iloc[i];
    experimentos.append([fila["dataset"], fila["n"], fila["R"], "PD", fila["archivo"]]);  

## Experimento :BT vs. PD

In [54]:
for i in range(0, df_fbps.shape[0]):
    fila = df_fbps.iloc[i];
    experimentos.append([fila["dataset"], fila["n"], fila["R"], "PD", fila["archivo"]]);
    experimentos.append([fila["dataset"], fila["n"], fila["R"], "BT", fila["archivo"]]);    

## Ejecutar los experimentos y guardar los resultados en un archivo CSV.
Este paso puede tardar unos minutos hasta terminar de ejecutarse.

In [82]:
columnas = ["dataset", "n", "R", "metodo", "tiempo"];
filas = [];
numero = 1
T = 5 # Numero de veces que se ejecuta cada experimento (para mayor fidelidad del tiempo).
for experimento in experimentos:
    # Voy mostrando que experimento se esta ejecutando.
    clear_output(wait=True)
    display('Experimento: ' + str(numero) + "/" + str(len(experimentos)))
    numero += 1
    
    # Ejecutamos el experimento T veces y obtenemos la mediana.
    tiempos = []
    resultados = []
    for i in range(0, T):
        t = correr_experimento(experimento[3], experimento[4])
        tiempos.append(t);
    tiempo = np.median(tiempos);
    filas.append([experimento[0], experimento[1], experimento[2], experimento[3], tiempo]);


'Experimento: 58/58'

In [56]:
df_resultados = pd.DataFrame(filas, columns=columnas);

df_resultados.to_csv("resultados/resultado.csv", index=False, header=True);

In [57]:
df_resultados

,dataset,n,R,metodo,tiempo
0,muchas-soluciones-validas,1,1000,FB,0.000972
1,muchas-soluciones-validas,2,1000,FB,0.001113
2,muchas-soluciones-validas,3,1000,FB,0.000979
3,muchas-soluciones-validas,4,1000,FB,0.001044
4,muchas-soluciones-validas,5,1000,FB,0.001243
...,...,...,...,...,...
7538,pocas-soluciones-validas,397,1000,BT,2.885890
7539,pocas-soluciones-validas,398,1000,PD,1.131720
7540,pocas-soluciones-validas,398,1000,BT,2.972470
7541,pocas-soluciones-validas,399,1000,PD,1.110550
